In [ ]:
from pynq.overlays.base import BaseOverlay
import cv2
import numpy as np
from pynq.lib.video import *
from time import sleep
import os
import argparse
import camera_configs  # 摄像头的标定数据
from my_hyperlpr import *
from PIL import Image,ImageDraw,ImageFont
import matplotlib.pyplot as plt
base = BaseOverlay("base.bit")

weightsPath="/home/xilinx/jupyter_notebooks/base/video/data/yolov3.weights"
configPath="/home/xilinx/jupyter_notebooks/base/video/data/yolov3.cfg"
labelsPath="/home/xilinx/jupyter_notebooks/base/video/data/coco.names"
rootdir = "/home/xilinx/jupyter_notebooks/base/video/data/photo_save/"   #图像读取地址
savepath = "/home/xilinx/jupyter_notebooks/base/video/data/images_save/"  # 图像保存地址

Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_RGB)
hdmi_out.start()

cap = cv2.VideoCapture(0)
# 设置摄像头分辨率 宽为1280，高为480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
print("Capture device is open: " + str(cap.isOpened()))

def show_chinese(imag,str,position,size,color):
    pil_img = cv2.cvtColor(imag,cv2.COLOR_BGR2RGB)#cv2和PIL中颜色的hex码的储存顺序不同，需转RGB模式
    pilimg = Image.fromarray(pil_img)#Image.fromarray()将数组类型转成图片格式，与np.array()相反
    draw = ImageDraw.Draw(pilimg)#PIL图片上打印汉字
    font = ImageFont.truetype("simhei.ttf",size,encoding="utf-8")
    #参数1：字体文件路径，参数2：字体大小；Windows系统“simhei.ttf”默认存储在路径：C:\Windows\Fonts中
    draw.text(position,str,color,font=font)
    cv2img = cv2.cvtColor(np.array(pilimg),cv2.COLOR_RGB2BGR)#将图片转成cv2.imshow()可以显示的数组格式
    return cv2img

def shot(pos, frame):
    #global i
    path = rootdir + pos + ".jpg"
    cv2.imwrite(path, frame[:,:,[2,1,0]])
    print("snapshot saved into: " + path)

def high_mea(x,y,threeD):
        if (x>639)or(y>480):
            return 0,0,0
        else:
            x1=(threeD[y][x][0])/200
            y1=(threeD[y][x][1])/200

        if abs(threeD[y][x][2]) < 1200000:
            dis=round((abs(threeD[y][x][2])/200),2)
            return x1,y1,1,dis
        else:
            return 0,0,0,0

freq = cv2.getTickFrequency()
font = cv2.FONT_HERSHEY_SIMPLEX

imageCount = 1    

LABELS = open(labelsPath).read().strip().split("\n")  #物体类别
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")#颜色

def aaa(frame1,frame2,limit):
    img1_rectified = cv2.remap(frame1, camera_configs.left_map1, camera_configs.left_map2, cv2.INTER_LINEAR,
                                   cv2.BORDER_CONSTANT)
    img2_rectified = cv2.remap(frame2, camera_configs.right_map1, camera_configs.right_map2, cv2.INTER_LINEAR,
                                   cv2.BORDER_CONSTANT)

    imgL = cv2.cvtColor(img1_rectified, cv2.COLOR_BGR2GRAY)
    imgR = cv2.cvtColor(img2_rectified, cv2.COLOR_BGR2GRAY)

    out = np.hstack((img1_rectified, img2_rectified))
    for i in range(0, out.shape[0], 30):
        cv2.line(out, (0, i), (out.shape[1], i), (0, 255, 0), 1)

    window_size = 3
    if imgL.ndim == 2:
        img_channels = 1
    else:
        img_channels = 3

    stereo = cv2.StereoSGBM_create(
            minDisparity=0,
            numDisparities=128,  # max_disp has to be dividable by 16 f. E. HH 192, 256
            blockSize=3,
            P1=8 * img_channels * window_size ** 2,
            # wsize default 3; 5; 7 for SGBM reduced size image; 15 for SGBM full size image (1300px and above); 5 Works nicely
            P2=32 * img_channels * window_size ** 2,
            disp12MaxDiff=1,
            uniquenessRatio=15,
            speckleWindowSize=200,
            speckleRange=1,
            preFilterCap=63,
            mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
        )
        # 对深度进行计算，获取深度矩阵
    disparity = stereo.compute(imgL, imgR).astype(np.float32) / 16.0

        # 按照深度矩阵生产深度图
    disp = cv2.normalize(disparity, disparity, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        # 将深度图扩展至三维空间中，其z方向的值则为当前的距离
    threeD = cv2.reprojectImageTo3D(disparity.astype(np.float32) / 16., camera_configs.Q)
            
    # 必须将boxes在遍历新的图片后初始化
    boxes = []
    confidences = []
    classIDs = []
    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
            #path = os.path.join(dirpath,filename)
    image = cv2.imread("./data/photo_save/left.jpg")
    (H, W) = image.shape[:2]
        # 得到 YOLO需要的输出层
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
            #从输入图像构造一个blob，然后通过加载的模型，给我们提供边界框和相关概率
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
            #在每层输出上循环
    for output in layerOutputs:
         #对每个检测进行循环
         for detection in output:
             scores = detection[5:]
             classID = np.argmax(scores)
             confidence = scores[classID]
                #过滤掉那些置信度较小的检测结果
             if confidence > 0.5:
                #框后接框的宽度和高度
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                #边框的左上角
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # 更新检测出来的框
                # 批量检测图片注意此处的boxes在每一次遍历的时候要初始化，否则检测出来的图像框会叠加
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
            # 极大值抑制
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.2,0.3)
    Y_high1=0
    Y_high2=0
    high=0
    ret_num1=0
    flag=0
    k=-1

    if len(idxs) > 0:
        for i in idxs.flatten() :
             (x, y) = (boxes[i][0], boxes[i][1])
             (w, h) = (boxes[i][2], boxes[i][3])
             print("坐标: "+str(x)+"_"+str(y)+"_"+str(x+w)+"_"+str(y+h))
             dis=0
             ret_num1=0
             Y_high1=0
             for num in range(1,100):
                 x_r=int(x+num*w/100)
                 y_r=y
                 X_r1,Y_r1,ret1,dis1=high_mea(x_r,y_r,threeD)
                 y_r=y+h
                 X_r2,Y_r2,ret2,dis2=high_mea(x_r,y_r,threeD)
                 if (ret1==1)and(ret2==1):
                     Y_high1=Y_high1+abs(Y_r1-Y_r2)
                     ret_num1=ret_num1+1
                     dis=dis+dis1+dis2
             if ret_num1==0 :
                print("无法测量")
             else :
                 high=Y_high1/ret_num1 
                 dis_f=dis/(2*ret_num1)
                 print("测量高度" +str(round(high,2))+"cm")
                 print("测量距离" +str(round(dis_f,2))+"cm")
                 print('\n')
                 if high>limit:
                     pr = LPR("models")            
                     res,left,top,right,bottom=pr.plate_recognition(frame1)
                     #print(left,top,right,bottom)
                     if right>=x & left<=x+w & top>=y & bottom<=y+h: 
                         cv2.rectangle(image, (x, y), (x + w, y + h), (0,0,255), 2)
                         image=show_chinese(image, "高度:"+str(round(high,2))+"cm", (x , y ),20,(0,0,255))
                         image=show_chinese(image, "距离:"+str(round(dis_f,2))+"cm", (x , y+20 ),20,(0,0,255))
                         image=show_chinese(image,"车牌:"+res,(x , y+40 ),20,(0,0,255))
                         flag=1
        if flag==0:
            image=show_chinese(image, "未检测到超高车辆", (0, 0),30,(0,0,255))
        else:
            image=show_chinese(image,"检测到超高车辆",(0 , 0 ),30,(0,0,255))
                
    cv2.imwrite(savepath+"total"+".jpg",image)
    return image

outframe = hdmi_out.newframe()
stage = 2
limit=190
while cap.isOpened():    
    if base.buttons[0].read()==1:
        stage=0
        sleep(0.1)
    if base.buttons[1].read()==1:
        stage=1
        sleep(0.1)
    if base.buttons[2].read()==1:
        limit=limit+5
        sleep(0.1)
    if base.buttons[3].read()==1:
        limit=limit-5
        sleep(0.1)
        
    if stage == 0:
        ret, frame = cap.read()
        outframe[:,:,:] = frame[:,0:640,:]
        hdmi_out.writeframe(outframe)
    if stage == 1:
        ret, frame = cap.read()
        left_img = frame[:, 0:640, :]
        right_img = frame[:, 640:1280, :]
        shot("left", left_img)
        shot("right", right_img)
        frame1 = cv2.imread("./data/photo_save/left.jpg")   
        frame2 = cv2.imread("./data/photo_save/right.jpg")  
        image=aaa(frame1,frame2,limit)
        outframe[:,:,:] = image[0:480,0:640,:]
        hdmi_out.writeframe(outframe)
        stage=4
    if stage == 2:
        img = cv2.imread("./data/main.jpg")
        cv2.putText(img, str(limit), (360, 420), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
        outframe[:,:,:] = img[:,:,:]
        hdmi_out.writeframe(outframe)
    if stage == 3:
        break
print("End" )
cap.release()

Capture device is open: True
坐标: 144_146_504_424
测量高度146.73cm
测量距离537.98cm


坐标: 1_201_72_255
无法测量
坐标: 511_168_631_224
测量高度79.45cm
测量距离870.32cm


坐标: 598_198_637_234
测量高度31.82cm
测量距离626.89cm


坐标: 178_80_473_353
测量高度187.68cm
测量距离471.86cm


坐标: -5_111_176_353
测量高度188.39cm
测量距离581.81cm


坐标: 498_126_638_336
测量高度114.79cm
测量距离394.18cm


